In [1]:
# Guy Baz
import openpyxl

In [2]:
wb = openpyxl.load_workbook(filename = "Sec-117-Public-Records-Complete-Oct-13-2023.xlsx")

In [3]:
ws = wb.active

In [4]:
# functions that support the code
def updateDictionary(Dict,item,amount):
    if amount is None:
        return Dict
        
    if item in Dict:
        Dict[item] += amount
    else:
        Dict[item] = amount
    return Dict

def WroteDictionaryToSheet(Dict,tit):
    # making a new sheet
    wsnew = wb.create_sheet()
    wsnew.title = tit
    wsnew.sheet_properties.tabColor = "e63019"
    wsnew['A1'] = "institution"
    wsnew['B1'] = "amount"
    a = 2
    for item in Dict:
        wsnew.cell(row = a, column = 1).value = item
        wsnew.cell(row = a, column = 2).value = Dict[item]
        a +=1
    return(wsnew)

def makePieChart(top,Sheet,tit):
    labels = openpyxl.chart.Reference(Sheet, min_col=1, min_row=2, max_row=(top+1))
    data = openpyxl.chart.Reference(Sheet, min_col=2, min_row=2, max_row=(top+1))
    pie = openpyxl.chart.PieChart()
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = tit
    Sheet.add_chart(pie, "D1")

In [5]:
# counting donations and aggregating by different variables - institution, state & donating country
institution = {}
states = {}
GiftOrigins = {}
# skip the first row as it is some title
# skip the second row as that includes the title of the columns
# Why the F column?
from openpyxl.utils.cell import column_index_from_string

for row in range(3,len(ws['F'])):
    amount = ws.cell(row, column_index_from_string('G')).value
    institute = ws.cell(row, column_index_from_string('B')).value
    state = ws.cell(row, column_index_from_string('C')).value
    Giver = ws.cell(row, column_index_from_string('F')).value
    institution = updateDictionary(institution,institute,amount)
    states = updateDictionary(states,state,amount)
    GiftOrigins = updateDictionary(GiftOrigins,Giver,amount)

In [6]:
# Taking top 10 from each category
# top_n_from_each_category(n)
top = 8
top_institution = sorted(institution, key=institution.get, reverse=True)[:top]
top_institution = {key: institution[key] for key in top_institution if key in institution}
top_states = sorted(states, key=states.get, reverse=True)[:top]
top_states = {key: states[key] for key in top_states if key in states}
top_GiftOrigins = sorted(GiftOrigins, key=GiftOrigins.get, reverse=True)[:top]
top_GiftOrigins = {key: GiftOrigins[key] for key in top_GiftOrigins if key in GiftOrigins}
print(top_institution)
print(top_states)
print(top_GiftOrigins)

{'Harvard University': 2829109124.7, 'Carnegie Mellon University': 2717299544.8, 'Cornell University': 2576291177.62, 'University of Pennsylvania': 2200357125.5999994, 'Massachusetts Institute of Technology': 1908071726.6599998, 'Stanford University': 1476558595.77, 'Johns Hopkins University': 1357575582.2800002, 'Yale University': 1263868324.3}
{'CA': 6898343877.793994, 'MA': 6172317400.259999, 'NY': 6057942086.640001, 'PA': 6043377719.38, 'TX': 3420765809.909999, 'IL': 2400097826.4900002, 'DC': 1686076307.06, 'MD': 1473375574.7800002}
{'QATAR': 5188401965.35, 'ENGLAND': 3723223015.9729996, 'CANADA': 3040620909.1500006, 'CHINA': 2910769845.0099993, 'SAUDI ARABIA': 2858152606.7339997, 'GERMANY': 2409069229.0599976, 'JAPAN': 2331532015.26, 'SWITZERLAND': 2137836295.6400008}


In [11]:
wb = openpyxl.Workbook()
(ws2) = WroteDictionaryToSheet(top_institution,"institutions")
makePieChart(top,ws2,"institutions")

(ws3) = WroteDictionaryToSheet(top_states,"states")
makePieChart(top,ws3,"states")

(ws4) = WroteDictionaryToSheet(top_GiftOrigins,"Donating Country")
makePieChart(top,ws4,"Donating Country")


In [12]:
wb.save("result.xlsx")